# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [38]:
city_weather = pd.read_csv(r"C:\Users\hmwhi\OneDrive\Desktop\GitHub\Python-API-challenge\WeatherPy\Outputs\city_data_df.csv")

city_weather_df = pd.DataFrame(city_weather)

city_weather_df = city_weather_df[[
    'City','City ID','Latitude','Longitude',
    'Temperature','Weather Description','Humidity',
    'Cloudiness', 'Wind Speed', 'Country'    
    ]]

city_weather_df.head()


,City,City ID,Latitude,Longitude,Temperature,Weather Description,Humidity,Cloudiness,Wind Speed,Country
0,Bluff,2008959,-46.6000,168.3333,45.806,broken clouds,51,54,3.601473,NZ
1,Los Llanos De Aridane,2002582,28.6585,-17.9182,67.118,clear sky,78,0,3.444888,ES
2,Busselton,2020551,-33.6500,115.3333,66.038,clear sky,67,0,8.008245,AU
3,Barrow,7649,71.2906,-156.7887,39.218,overcast clouds,87,90,18.410016,US
4,Ushuaia,8303,-54.8000,-68.3000,40.658,scattered clouds,75,40,6.912145,AR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
coordinates = city_weather_df[['Latitude','Longitude']]
humidity = city_weather_df['Humidity']

gmaps.configure(api_key= gkey)


In [40]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=.9,center=(30,10))

humid_layer = gmaps.heatmap_layer(coordinates,weights=humidity,
                                dissipating=False,max_intensity=100,point_radius=1)

fig.add_layer(humid_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
good_weather = city_weather_df.loc[city_weather_df['Weather Description'] == 'clear sky']

good_weather = good_weather.loc[good_weather['Cloudiness'] == 0]
good_weather = good_weather.loc[good_weather['Wind Speed'] < 10]
good_weather = good_weather.loc[good_weather['Temperature'] < 80]
good_weather = good_weather.loc[good_weather['Temperature'] > 70]

# print(len(good_weather['City']))

good_weather = good_weather.dropna()
good_weather.head()

print(len(good_weather['City']))

good_weather.head()



18


,City,City ID,Latitude,Longitude,Temperature,Weather Description,Humidity,Cloudiness,Wind Speed,Country
47,Geraldton,9522,-28.7667,114.6000,70.214,clear sky,40,0,3.444888,AU
117,Waqqas,6862,32.5427,35.6053,71.600,clear sky,61,0,3.042238,JO
150,Qiryat Yam,6853,32.8508,35.0703,75.146,clear sky,57,0,2.304048,IL
188,Teguise,2379,29.0605,-13.5640,70.016,clear sky,83,0,5.748936,ES
234,Arrecife,2379,28.9630,-13.5477,73.382,clear sky,83,0,5.748936,ES


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
import json as json

hotel_df =  good_weather
hotel_df['Hotel Name'] = ""

params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# for loop where coords are coordinates from the good weather df
for index,row in hotel_df.iterrows():
    # location = f'{lat},{lng}'
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']

    params['location'] = f"{lat},{lng}"
    # print(params)

    # query_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=hotel&location={lat}%2C{lng}&radius=1500&type=lodging&key={gkey}'
    
    response = requests.get(base_url,params=params)
    # print(response)
    hotel_location = response.json()
    # print(hotel_location)

    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_location['results'][0]['name']
        # hotel_df[index,'Hotel Name'] = hotel_location['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


hotel_df.head()


Missing field/result... skipping.


,City,City ID,Latitude,Longitude,Temperature,Weather Description,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
47,Geraldton,9522,-28.7667,114.6000,70.214,clear sky,40,0,3.444888,AU,Broadwater Mariner Resort
117,Waqqas,6862,32.5427,35.6053,71.600,clear sky,61,0,3.042238,JO,Jordan EcoPark
150,Qiryat Yam,6853,32.8508,35.0703,75.146,clear sky,57,0,2.304048,IL,Sweet home
188,Teguise,2379,29.0605,-13.5640,70.016,clear sky,83,0,5.748936,ES,B&B La Casona de Nazaret
234,Arrecife,2379,28.9630,-13.5477,73.382,clear sky,83,0,5.748936,ES,Hotel Lancelot


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[info_box_template.format(**row)for index,
    row in hotel_df.iterrows()])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…